### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [2]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [3]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidfvect = TfidfVectorizer()

In [7]:
# en el atributo `data` accedemos al texto
print(newsgroups_train.data[0])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


In [8]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [9]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [10]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [11]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [12]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [13]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [14]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [15]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [16]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ], shape=(11314,))

In [17]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 4811,  6635,  4253, ...,  1534, 10055,  4750], shape=(11314,))

In [18]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [19]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [20]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [21]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

,alpha,1.0
,force_alpha,True
,fit_prior,True
,class_prior,None


In [22]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [23]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Construir un modelo de clasificación por prototipos (tipo zero-shot). Clasificar los documentos de un conjunto de test comparando cada uno con todos los de entrenamiento y asignar la clase al label del documento del conjunto de entrenamiento con mayor similaridad.

**3**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**4**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


### Comparación de documentos

In [27]:
np.random.seed(1)  # Para reproducibilidad
random_docs = np.random.choice(len(newsgroups_train.data), 5, replace=False)

print("Documentos seleccionados al azar:")
print(random_docs)

Documentos seleccionados al azar:
[11200  5836  4362  5309  4422]


In [30]:
print(f"\nDocumento 11200:")
print(f"Categoría: {newsgroups_train.target_names[y_train[11200]]}")
print(f"Texto (primeros 400 caracteres): {newsgroups_train.data[11200][:400]}...")

doc_similarity = cosine_similarity(X_train[11200], X_train)[0]

most_similar_indices = np.argsort(doc_similarity)[::-1][1:6]

print(f"\nLos 5 documentos más similares:")
for j, sim_idx in enumerate(most_similar_indices):
    similarity_score = doc_similarity[sim_idx]
    category = newsgroups_train.target_names[y_train[sim_idx]]
    print(f"\n  {j+1}. Índice: {sim_idx}, Similaridad: {similarity_score:.4f}, Categoría: {category}")
    print(f"     Texto (primeros 300 caracteres): {newsgroups_train.data[sim_idx][:300]}...")


Documento 11200:
Categoría: rec.motorcycles
Texto (primeros 400 caracteres): Does anyone have a rear wheel for a PD they'd like to part with?

Does anyone know where I might find one salvage?

As long as I'm getting the GIVI luggage for Brunnhilde and have
the room, I thought I'd carry a spare.

Ride Free,

Bill
___________________________________________________________________             
johnsw@wsuvm1.csc.wsu.edu  prez=BIMC  KotV KotRR                                
D...

Los 5 documentos más similares:

  1. Índice: 10480, Similaridad: 0.1995, Categoría: sci.med
     Texto (primeros 300 caracteres): 

does anyone know?

-- ...

  2. Índice: 10966, Similaridad: 0.1772, Categoría: talk.politics.misc
     Texto (primeros 300 caracteres): does anyone have Prez. Clinton`s e-mail address.
thanks a lot 
 


...

  3. Índice: 4930, Similaridad: 0.1298, Categoría: rec.motorcycles
     Texto (primeros 300 caracteres): This is a periodic posting intended to answer the Frequently Asked
Ques

En este primer caso vemos que las categorías casi nunca coinciden, pero si se puede ver similaridad en los textos. Por ejemplo, las palabras "Does" y "Anyone" se repiten el los dos documentos con más similaridad al de pruebas. Como son documentos cortos, solo tenemos esa información para compararlos y terminan siendo similares a pesar de sus diferencias categóricas.


In [32]:
print(f"\nDocumento 5836:")
print(f"Categoría: {newsgroups_train.target_names[y_train[5836]]}")
print(f"Texto (primeros 400 caracteres): {newsgroups_train.data[5836][:400]}...")

doc_similarity = cosine_similarity(X_train[5836], X_train)[0]

most_similar_indices = np.argsort(doc_similarity)[::-1][1:6]

print(f"\nLos 5 documentos más similares:")
for j, sim_idx in enumerate(most_similar_indices):
    similarity_score = doc_similarity[sim_idx]
    category = newsgroups_train.target_names[y_train[sim_idx]]
    print(f"\n  {j+1}. Índice: {sim_idx}, Similaridad: {similarity_score:.4f}, Categoría: {category}")
    print(f"     Texto (primeros 300 caracteres): {newsgroups_train.data[sim_idx][:300]}...")


Documento 5836:
Categoría: talk.religion.misc
Texto (primeros 400 caracteres): What are the consequences of the homophobic ranting of the
self-righteous?  Well, I just noted this on another group,
and thought I'd pass it along.  The context is talk.origins,
and a report of yet another "debate" that was nothing but an
attempt at mindless bullying and factless assertion by a
standard-issue Creationist.  The writer reflects that the
behavior reported reminds him of some Christi...

Los 5 documentos más similares:

  1. Índice: 5826, Similaridad: 0.3325, Categoría: soc.religion.christian
     Texto (primeros 300 caracteres): A listmember (D Andrew Killie, I think) wrote, in response to the
suggestion that genocide may sometimes be the will of God:

 > Any God who works that way is indescribably evil,
 > and unworthy of my worship or faith.

Nobuya "Higgy" Higashiyama replied (as, in substance, did others):

 > Where is ...

  2. Índice: 10836, Similaridad: 0.3325, Categoría: alt.atheism
 

En este caso todas las categorías están relacionadas por más que no son exactamente las mismas. La categoría "religion misc" probablemente comparte muchas palabras en común como "god" con las categorías "religion atheism", "religion christian" y "politics mideast" (ya que la política en el medio oriente generalmente involucra temas religiosos).

In [33]:
print(f"\nDocumento 4362:")
print(f"Categoría: {newsgroups_train.target_names[y_train[4362]]}")
print(f"Texto (primeros 400 caracteres): {newsgroups_train.data[4362][:400]}...")

doc_similarity = cosine_similarity(X_train[4362], X_train)[0]

most_similar_indices = np.argsort(doc_similarity)[::-1][1:6]

print(f"\nLos 5 documentos más similares:")
for j, sim_idx in enumerate(most_similar_indices):
    similarity_score = doc_similarity[sim_idx]
    category = newsgroups_train.target_names[y_train[sim_idx]]
    print(f"\n  {j+1}. Índice: {sim_idx}, Similaridad: {similarity_score:.4f}, Categoría: {category}")
    print(f"     Texto (primeros 300 caracteres): {newsgroups_train.data[sim_idx][:300]}...")


Documento 4362:
Categoría: soc.religion.christian
Texto (primeros 400 caracteres): A "new Christian" wrote that he was new to the faith and 
learning about it "by reading the Bible, of course". I am not
at all sure this is the best path to follow.
	While the Bible is, for Christians, the word of God, the 
revelation of God is Jesus Christ and the chief legacy of this
revalation is the Church. I am not recommending any one
denommination, but I do recommend finding a comfortable c...

Los 5 documentos más similares:

  1. Índice: 2772, Similaridad: 0.4560, Categoría: soc.religion.christian
     Texto (primeros 300 caracteres): 

The problem you see here is that some Christians claim things about
the Bible which they don't actually believe or practice. I've known
all sorts of Christians, ranging from the trendiest of liberals to
the fire-breathing fundamentalists, and although many on the 
conservative side of the Christia...

  2. Índice: 9914, Similaridad: 0.4464, Categoría: soc.religi

En este caso sucede algo similar a lo anterior, resaltando palabras como "christian" que aparecen en varios documentos relacionados con la religión, especialmente en los contextos de "religion christian" y "religion misc".


In [34]:
print(f"\nDocumento 5309:")
print(f"Categoría: {newsgroups_train.target_names[y_train[5309]]}")
print(f"Texto (primeros 400 caracteres): {newsgroups_train.data[5309][:400]}...")

doc_similarity = cosine_similarity(X_train[11200], X_train)[0]

most_similar_indices = np.argsort(doc_similarity)[::-1][1:6]

print(f"\nLos 5 documentos más similares:")
for j, sim_idx in enumerate(most_similar_indices):
    similarity_score = doc_similarity[sim_idx]
    category = newsgroups_train.target_names[y_train[sim_idx]]
    print(f"\n  {j+1}. Índice: {sim_idx}, Similaridad: {similarity_score:.4f}, Categoría: {category}")
    print(f"     Texto (primeros 300 caracteres): {newsgroups_train.data[sim_idx][:300]}...")


Documento 5309:
Categoría: talk.religion.misc
Texto (primeros 400 caracteres): 
Glad to hear this, just a note, Osiris, Mithras and many other
cult gods resurrected as well, so there's a good chance for all of
us to maybe end up in a virtual reality simulator, and live forever,
hurrah!

Sorry, this was a joke, some sort of one anyway. I'm the first
that connected Osiris with a virtual reality personality database.
Time to write a book.


Cheers,
Kent...

Los 5 documentos más similares:

  1. Índice: 10480, Similaridad: 0.1995, Categoría: sci.med
     Texto (primeros 300 caracteres): 

does anyone know?

-- ...

  2. Índice: 10966, Similaridad: 0.1772, Categoría: talk.politics.misc
     Texto (primeros 300 caracteres): does anyone have Prez. Clinton`s e-mail address.
thanks a lot 
 


...

  3. Índice: 4930, Similaridad: 0.1298, Categoría: rec.motorcycles
     Texto (primeros 300 caracteres): This is a periodic posting intended to answer the Frequently Asked
Question: What is the DoD? 

En este caso podemos ver casos con valores de similaridad muy bajos, probablemente porque el documento a comparar es muy diferente en contenido a los demás documentos del conjunto y no comparte demasiadas palabras clave.

In [35]:
print(f"\nDocumento 4422:")
print(f"Categoría: {newsgroups_train.target_names[y_train[4422]]}")
print(f"Texto (primeros 400 caracteres): {newsgroups_train.data[4422][:400]}...")

doc_similarity = cosine_similarity(X_train[4422], X_train)[0]

most_similar_indices = np.argsort(doc_similarity)[::-1][1:6]

print(f"\nLos 5 documentos más similares:")
for j, sim_idx in enumerate(most_similar_indices):
    similarity_score = doc_similarity[sim_idx]
    category = newsgroups_train.target_names[y_train[sim_idx]]
    print(f"\n  {j+1}. Índice: {sim_idx}, Similaridad: {similarity_score:.4f}, Categoría: {category}")
    print(f"     Texto (primeros 300 caracteres): {newsgroups_train.data[sim_idx][:300]}...")


Documento 4422:
Categoría: talk.politics.guns
Texto (primeros 400 caracteres): NUT CASE PANICS!!!!JUMPS THE GUN ON THE NET BEFORE GETTING FACTS STRAIGHT!!!!
...

Los 5 documentos más similares:

  1. Índice: 10846, Similaridad: 0.3014, Categoría: talk.politics.guns
     Texto (primeros 300 caracteres): NUT CASE PANICS!!!!REALIZES HE'S MADE A COMPLETE FOOL OF HIMSELF IN FRONT OF
THOUSANDS OF NETTERS!!!!BACKS AWAY FROM EARLIER RASH STATEMENTS!!!!GOD HAVE
MERCY ON HIM!!!!
...

  2. Índice: 6704, Similaridad: 0.1589, Categoría: sci.space
     Texto (primeros 300 caracteres): Okay, lets get the record straight on the Livermore gas gun.  
The project manager is Dr. John Hunter, and he works for the
Laser group at Livermore.  What, you may ask, does gas guns
have to do with lasers? Nothing, really, but the gun is physically
located across the road from the Free Electron La...

  3. Índice: 11036, Similaridad: 0.1429, Categoría: talk.politics.guns
     Texto (primeros 300 caracteres): 
	At th

En este caso de comparación final, podemos ver que la categoría original "Politics Guns" tiene una palabra muy particular que seguramente aparezca en la mayoría de los casos, "guns". Es por eso que la mayoría de las similitudes encontradas están relacionadas con esta palabra clave, lo que indica que los documentos que comparten esta palabra tienden a ser más similares entre sí.

### Clasificación zero-shot

In [36]:
# Modelo de clasificación por prototipos (zero-shot)
# Clasificamos cada documento de test comparando con todos los de train
# y asignando la clase del documento más similar

def prototype_classify(X_train, y_train, X_test):
    predictions = []
    
    for i in range(X_test.shape[0]):
        # Calcular similaridad coseno entre el documento de test y todos los de train
        similarities = cosine_similarity(X_test[i], X_train)[0]
        
        # Encontrar el índice del documento más similar
        most_similar_idx = np.argmax(similarities)
        
        # Asignar la clase del documento más similar
        predicted_class = y_train[most_similar_idx]
        predictions.append(predicted_class)
        
        # Mostrar progreso cada 1000 documentos
        if (i + 1) % 1000 == 0:
            print(f"Procesados {i + 1}/{X_test.shape[0]} documentos")
    
    return np.array(predictions)

# Ejecutar la clasificación por prototipos
y_pred_prototype = prototype_classify(X_train, y_train, X_test)

Procesados 1000/7532 documentos
Procesados 2000/7532 documentos
Procesados 3000/7532 documentos
Procesados 4000/7532 documentos
Procesados 5000/7532 documentos
Procesados 6000/7532 documentos
Procesados 7000/7532 documentos


In [40]:
f1_prototype = f1_score(y_test, y_pred_prototype, average='macro')
print(f"F1-score macro del modelo de prototipos: {f1_prototype:.4f}")

f1_nb = f1_score(y_test, y_pred, average='macro')
print(f"F1-score macro del modelo Naive Bayes: {f1_nb:.4f}")

print(f"\nDiferencia: {f1_nb - f1_prototype:.4f}")

F1-score macro del modelo de prototipos: 0.5050
F1-score macro del modelo Naive Bayes: 0.5854

Diferencia: 0.0804


### Naive Bayes mejorado

In [41]:
vectorizer_configs = [
    {'max_features': 10000, 'min_df': 2, 'max_df': 0.95, 'ngram_range': (1, 1)},
    {'max_features': 15000, 'min_df': 2, 'max_df': 0.95, 'ngram_range': (1, 2)},
    {'max_features': 20000, 'min_df': 3, 'max_df': 0.9, 'ngram_range': (1, 1)},
    {'max_features': None, 'min_df': 5, 'max_df': 0.8, 'ngram_range': (1, 2)}
]

models = {
    'MultinomialNB': MultinomialNB(),
    'ComplementNB': ComplementNB()
}

best_f1 = 0
best_config = None
best_model_name = None
best_vectorizer = None
best_model = None

for i, config in enumerate(vectorizer_configs):
    print(f"\nConfiguración {i+1}: {config}")
    
    # Crear nuevo vectorizador con la configuración actual
    vectorizer = TfidfVectorizer(**config)
    
    # Vectorizar los datos
    X_train_config = vectorizer.fit_transform(newsgroups_train.data)
    X_test_config = vectorizer.transform(newsgroups_test.data)
    
    # Probar ambos modelos
    for model_name, model in models.items():
        # Entrenar el modelo
        model.fit(X_train_config, y_train)
        
        # Predecir en test
        y_pred_config = model.predict(X_test_config)
        
        # Calcular F1-score
        f1_config = f1_score(y_test, y_pred_config, average='macro')
        
        print(f"  {model_name}: F1-score = {f1_config:.4f}")
        
        # Guardar si es el mejor resultado
        if f1_config > best_f1:
            best_f1 = f1_config
            best_config = config
            best_model_name = model_name
            best_vectorizer = vectorizer
            best_model = model

print(f"\n{'='*50}")
print(f"MEJOR RESULTADO:")
print(f"Modelo: {best_model_name}")
print(f"F1-score: {best_f1:.4f}")
print(f"Configuración del vectorizador: {best_config}")
print(f"{'='*50}")


Configuración 1: {'max_features': 10000, 'min_df': 2, 'max_df': 0.95, 'ngram_range': (1, 1)}
  MultinomialNB: F1-score = 0.6156
  ComplementNB: F1-score = 0.6668

Configuración 2: {'max_features': 15000, 'min_df': 2, 'max_df': 0.95, 'ngram_range': (1, 2)}
  MultinomialNB: F1-score = 0.5853
  ComplementNB: F1-score = 0.6520

Configuración 3: {'max_features': 20000, 'min_df': 3, 'max_df': 0.9, 'ngram_range': (1, 1)}
  MultinomialNB: F1-score = 0.6083
  ComplementNB: F1-score = 0.6862

Configuración 4: {'max_features': None, 'min_df': 5, 'max_df': 0.8, 'ngram_range': (1, 2)}
  MultinomialNB: F1-score = 0.5813
  ComplementNB: F1-score = 0.6840

MEJOR RESULTADO:
Modelo: ComplementNB
F1-score: 0.6862
Configuración del vectorizador: {'max_features': 20000, 'min_df': 3, 'max_df': 0.9, 'ngram_range': (1, 1)}


### Matriz término-documento y similaridad entre palabras

In [42]:
X_term_doc = X_train.T

print(f"Matriz documento-término original: {X_train.shape}")
print(f"Matriz término-documento transpuesta: {X_term_doc.shape}")

palabras_seleccionadas = ['computer', 'car', 'god', 'gun', 'space']

print(f"\nPalabras seleccionadas para análisis: {palabras_seleccionadas}")

word_indices = {}
for palabra in palabras_seleccionadas:
    if palabra in tfidfvect.vocabulary_:
        word_indices[palabra] = tfidfvect.vocabulary_[palabra]
        print(f"'{palabra}' encontrada en índice: {word_indices[palabra]}")
    else:
        print(f"'{palabra}' NO encontrada en el vocabulario")

print("\n" + "="*60)

for palabra, idx in word_indices.items():
    print(f"\nAnálisis de similaridad para '{palabra}':")
    
    word_similarities = cosine_similarity(X_term_doc[idx], X_term_doc)[0]

    # Obtener los 5 más similares (excluyendo la palabra misma)
    most_similar_word_indices = np.argsort(word_similarities)[::-1][1:6]
    
    print(f"Las 5 palabras más similares a '{palabra}':")
    for i, similar_idx in enumerate(most_similar_word_indices):
        similarity_score = word_similarities[similar_idx]
        similar_word = idx2word[similar_idx]
        print(f"  {i+1}. '{similar_word}' - Similaridad: {similarity_score:.4f}")

Matriz documento-término original: (11314, 101631)
Matriz término-documento transpuesta: (101631, 11314)

Palabras seleccionadas para análisis: ['computer', 'car', 'god', 'gun', 'space']
'computer' encontrada en índice: 28940
'car' encontrada en índice: 25775
'god' encontrada en índice: 43842
'gun' encontrada en índice: 44820
'space' encontrada en índice: 84097


Análisis de similaridad para 'computer':
Las 5 palabras más similares a 'computer':
  1. 'decwriter' - Similaridad: 0.1563
  2. 'harkens' - Similaridad: 0.1522
  3. 'deluged' - Similaridad: 0.1522
  4. 'shopper' - Similaridad: 0.1443
  5. 'the' - Similaridad: 0.1361

Análisis de similaridad para 'car':
Las 5 palabras más similares a 'car':
  1. 'cars' - Similaridad: 0.1797
  2. 'criterium' - Similaridad: 0.1770
  3. 'civic' - Similaridad: 0.1748
  4. 'owner' - Similaridad: 0.1689
  5. 'dealer' - Similaridad: 0.1681

Análisis de similaridad para 'god':
Las 5 palabras más similares a 'god':
  1. 'jesus' - Similaridad: 0.2688
  2

Analizando la similaridad de las palabras elegidas, se puede observar que en la mayoría de los casos las palabras están relacionadas y pueden considerarse que son inherentes al mismo contexto temático. Esto sugiere que el modelo ha capturado adecuadamente las relaciones semánticas entre los términos, lo cual es un indicativo positivo de su desempeño en tareas de clasificación de texto.